In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import glob
import math
import keras
from keras import models
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras import metrics
from keras import regularizers
from keras.layers import Dropout
from keras.constraints import maxnorm

Using TensorFlow backend.
/home/varshi/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/varshi/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/varshi/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/varshi/.local/lib/python3.7/site-packages/tensorflow/python

In [5]:
train_val_set = pd.read_csv("Desktop/Project/TrainingData.csv")
test_set = pd.read_csv("Desktop/Project/ValidationData.csv")

In [6]:
train_val_set.iloc[:, 0:520].min().min() # minimum WAP is -104 dBm
train_val_set_P = train_val_set.copy()
train_val_set_P.iloc[:, 0:520] = np.where(train_val_set_P.iloc[:, 0:520] <= 0, 
                train_val_set_P.iloc[:, 0:520] + 105, 
                train_val_set_P.iloc[:, 0:520] - 100) 

In [7]:
train_val_set_PN = train_val_set_P.copy()
train_val_set_PN.iloc[:, 0:520].max().max()
train_val_set_PN.iloc[:, 0:520] = train_val_set_P.iloc[:, 0:520]/105

In [10]:
combined = pd.concat([train_val_set_PN, test_set]) # stack vertically
combined = combined.assign(NEWID= (combined['LONGITUDE'].astype(str) + '_' + combined['LATITUDE'].astype(str) + '_' + combined['FLOOR'].astype(str) + '_' + combined['BUILDINGID'].astype(str)).astype('category').cat.codes)

In [14]:
train_val_set_PUN = combined.iloc[0:19937, :]
test_set_U = combined.iloc[19937:21048, :]

In [15]:
train_val_set_PUN["UNIQUELOCATION"] = train_val_set_PUN["UNIQUELOCATION"].astype("category")

KeyError: 'UNIQUELOCATION'

In [10]:
dummy = keras.utils.to_categorical(train_val_set_PUN['UNIQUELOCATION'], num_classes = 1995)
dummy = pd.DataFrame(dummy, dtype = 'int')
train_val_set_PUND = pd.concat([train_val_set_PUN, dummy] ,axis = 1)


X_train_val = train_val_set_PUND.iloc[:, 0:520]
y_train_val = train_val_set_PUND.iloc[:, 520:2525]

In [11]:
test_set_PU = test_set_U.copy()
test_set_PU.iloc[:, 0:520] = np.where(test_set_PU.iloc[:, 0:520] <= 0, test_set_PU.iloc[:, 0:520] + 105, test_set_PU.iloc[:, 0:520] - 100) 
test_set_PUN = test_set_PU.copy()
test_set_PUN.iloc[:, 0:520].max().max()
test_set_PUN.iloc[:, 0:520] = test_set_PU.iloc[:, 0:520]/105

In [12]:
test_set_PUN["UNIQUELOCATION"] = test_set_PUN["UNIQUELOCATION"].astype("category")
test_set_PUN.dtypes

WAP001               float64
WAP002               float64
WAP003               float64
WAP004               float64
WAP005               float64
                      ...   
RELATIVEPOSITION       int64
USERID                 int64
PHONEID                int64
TIMESTAMP              int64
UNIQUELOCATION      category
Length: 530, dtype: object

In [13]:
dummy = keras.utils.to_categorical(test_set_PUN['UNIQUELOCATION'], num_classes = 1995)
dummy = pd.DataFrame(dummy, dtype = 'int')
test_set_PUND = pd.concat([test_set_PUN, dummy] ,axis = 1)


X_test = test_set_PUND.iloc[:, 0:520]
y_test = test_set_PUND.iloc[:, 520:2525]

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, 
                                                  test_size = 0.2, 
                                                  random_state = 0)


In [1]:
hyperparameters = {'epochs': 150, # SGD hyperparameter
                   'batch_size': 800, # SGD hyperparameter
                   'hidden_layers': 1,
                   'neurons': 1600, # neurons per hidden layer
                   'lambd': None, # L2 regularization term
                   'dropout': None} # dropout probability for dropout regularization

In [2]:
def create_classifier(num_features, num_classes, hidden_layers = 3, 
                      neurons = 1600, lambd = 0, dropout = 0): 
    
    classifier = Sequential() 
    
    classifier.add(Dense(input_dim = num_features, 
                         units = neurons, 
                         kernel_initializer = 'glorot_uniform',
                         #kernel_regularizer=regularizers.l2(lambd),
                         activation = 'relu'))
                         
    for i in range(hidden_layers - 1):
        classifier.add(Dense(units = neurons, 
                     kernel_initializer = 'glorot_uniform',
                     #kernel_regularizer=regularizers.l2(lambd),
                     activation = 'relu'))
    classifier.add(Dense(units = num_classes, 
                         kernel_initializer = 'glorot_uniform', 
                         activation='softmax'))
    
    # compile classifier 
    classifier.compile(loss = 'categorical_crossentropy', 
                       optimizer = 'adam', 
                       metrics = [metrics.categorical_accuracy])
    
    classifier.summary()
    return classifier

In [3]:
classifier = create_classifier(num_features = X_train.shape[1],
                               num_classes = 1995,
                               hidden_layers = hyperparameters['hidden_layers'],
                               neurons = hyperparameters['neurons'],
                               lambd = hyperparameters['lambd'])

classifier.fit(X_train, y_train.iloc[:, 10:2005], batch_size = hyperparameters['batch_size'], epochs = hyperparameters['epochs'], 
                 verbose = True, 
                 shuffle = True) 

NameError: name 'X_train' is not defined

In [36]:
train_results = classifier.evaluate(X_train, y_train.iloc[:, 10:2005])
print(train_results)

15949/15949 [==============================] - 5s 341us/step
[0.21926532620682643, 0.9457646012306213]


In [35]:
val_results = classifier.evaluate(X_val, y_val.iloc[:, 10:2005], batch_size = 5)
print(val_results)

3988/3988 [==============================] - 6s 2ms/step
[0.7078392913374362, 0.7918756008148193]


In [19]:
y_pred = np.argmax(classifier.predict(X_test), axis = 1)

In [20]:
y_pred

array([1063,  526,  409, ..., 1552, 1651, 1552])

In [28]:
model = models.save_model(classifier,"Indoor.h5")